# Demo - Annual Reports

* Read page from Annual Report of Rabobank and ING
* Prompt LLM to Extract financial KPIs
* output as JSON to create a DataFrame
* Compare results for the two reports

### Sources:
* https://learn.deeplearning.ai/langchain
* https://python.langchain.com/docs/get_started/quickstart

### Requirements

In [1]:
!pip install pypdf langchain openai langchain-openai neo4j python-dotenv langchainhub langchain-community --quiet

In [2]:
%load_ext watermark
%watermark -p langchain,langchainhub,langchain_community,pypdf

langchain          : 0.1.5
langchainhub       : 0.1.14
langchain_community: 0.0.17
pypdf              : 4.0.1



### Imports

In [3]:
import os
from graphdatascience import GraphDataScience
from dotenv import load_dotenv, find_dotenv, dotenv_values
from pathlib import Path
import neo4j
import json

from langchain_openai import ChatOpenAI

from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.tools import Tool

from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain

from langchain import hub
from langchain_community.document_loaders import PyPDFLoader

### Settings

In [4]:
project_path = Path(os.getcwd()).parent
data_path = project_path / "data"
model_path = project_path / "models"
output_path = project_path / "output"

llm_model = "gpt-4"

# load env settings
load_dotenv("../.env.lotr")

openai_api_key = os.getenv('OPENAI_API_KEY')

### 1. Read pdf

* Picked PyMyPDF because of this article: https://medium.com/social-impact-analytics/comparing-4-methods-for-pdf-text-extraction-in-python-fd34531034f

In [5]:
os.listdir(data_path)

['JRR Tolkien - Lord of the Rings Collection.pdf',
 '2023-ING-Groep-NV-annual-report.pdf',
 'Annual Report 2023.pdf',
 'recommendations-50.dump']

In [6]:
filename = "Annual Report 2023.pdf"

In [7]:
%%time
# Wall time: 4.18 s

loader = PyPDFLoader(str(data_path / filename))
pages = loader.load_and_split()

CPU times: user 13.9 s, sys: 60.8 ms, total: 13.9 s
Wall time: 13.9 s


### 2. Extract financial KPIs from selected pages 

In [8]:
selected_page_numbers = [50, 51, 52]
selected_pages = [page for page in pages if page.metadata['page'] in selected_page_numbers]

In [9]:
%%time

llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0)
print("llm.temperature:", llm.temperature)

prompt = PromptTemplate(template="""Extract financial KPIs out this text from an annual report and return them as as list of key value pairs

<context>
{context}
</context>

In addition, provide a summary for the text and estimate wheter the performance is better or worse than expected if you can find this from the text.

{output}
l""", input_variables=["context", "output"])


document_chain = create_stuff_documents_chain(llm, prompt)

response = document_chain.invoke({
    "context": [selected_pages[1]], "output": "Output the response as valid JSON!"
})
print(response)

llm.temperature: 0.0
{
    "financial_KPIs": {
        "Net Interest Income": "EUR 11,712 million",
        "Net Fee and Commission Income": "EUR 2,091 million",
        "Other Results": "EUR 1,602 million",
        "Operating Expenses": "EUR 8,057 million",
        "Operating Profit before tax": "EUR 5,962 million",
        "Taxes": "EUR 1,585 million"
    },
    "summary": "The financial performance of Rabobank in 2023 showed improvements in key financial indicators such as Net Interest Income, Net Fee and Commission Income, and Other Results. Operating Expenses increased due to higher staff costs. Operating Profit before tax also increased significantly compared to the previous year. Overall, the performance seems better than expected.",
    "performance": "better"
}
CPU times: user 35.1 ms, sys: 15.4 ms, total: 50.5 ms
Wall time: 3.63 s


### 3. parse response as json and create dataframe

In [10]:
response_json = json.loads(response)

import pandas as pd
display(pd.DataFrame([response_json['financial_KPIs']]).T)
print(response_json['summary'])

,0
Net Interest Income,"EUR 11,712 million"
Net Fee and Commission Income,"EUR 2,091 million"
Other Results,"EUR 1,602 million"
Operating Expenses,"EUR 8,057 million"
Operating Profit before tax,"EUR 5,962 million"
Taxes,"EUR 1,585 million"


The financial performance of Rabobank in 2023 showed improvements in key financial indicators such as Net Interest Income, Net Fee and Commission Income, and Other Results. Operating Expenses increased due to higher staff costs. Operating Profit before tax also increased significantly compared to the previous year. Overall, the performance seems better than expected.


### 4. Read multiple reports

In [11]:
annual_reports = [
    {'company': 'Rabobank', 'filename': 'Annual Report 2023.pdf', 'selected_pages': [50, 51, 52]},
    {'company': 'ING', 'filename': '2023-ING-Groep-NV-annual-report.pdf', 'selected_pages': [207, 208, 209]}
]

In [12]:
%%time

for report in annual_reports:
    print(report['filename'])
    
    # load pdf
    loader = PyPDFLoader(str(data_path / report['filename']))
    report['pages'] = loader.load_and_split()

Annual Report 2023.pdf
2023-ING-Groep-NV-annual-report.pdf
CPU times: user 27.2 s, sys: 95.1 ms, total: 27.3 s
Wall time: 27.3 s


#### 4.1 Response as text

In [13]:
llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0)
print("llm.temperature:", llm.temperature)

prompt = PromptTemplate(template="""Extract financial KPIs out this text from an annual report and return them as as list of key value pairs

<context>
{context}
</context>

In addition, provide a summary for the text and estimate wheter the performance is better or worse than expected if you can find this from the text.

{output}
l""", input_variables=["context", "output"])

llm.temperature: 0.0


#### 4.2 Parse selected pages

In [14]:
%%time

document_chain = create_stuff_documents_chain(llm, prompt)
document_json_chain = create_stuff_documents_chain(llm, prompt, output_parser=SimpleJsonOutputParser())

for report in annual_reports:
    print("===")
    print(report['filename'])
    
    output = []
    output_json = []
    for i, page in enumerate(report['pages']):
        if page.metadata['page'] in report['selected_pages']:
            print(page.metadata)
            
            response = document_chain.invoke({
                "context": [page], "output": ""
            })

            output.append({'page_index': i, 'page': page.metadata['page'], 'response': response})
            
            try:
                # as valid JSON
                response_json = document_json_chain.invoke({
                    "context": [page], "output": "Very important is that you output the response as valid JSON!"
                })
                output_json.append({'page_index': i, 'page': page.metadata['page'], 'response': response_json})
            except Exception as e:
                print("error getting json, skip page")
        
    report['responses'] = output
    report['responses_json'] = output_json

===
Annual Report 2023.pdf
{'source': '/Users/stetelepta/work/neo4j-llm/data/Annual Report 2023.pdf', 'page': 50}
{'source': '/Users/stetelepta/work/neo4j-llm/data/Annual Report 2023.pdf', 'page': 51}
{'source': '/Users/stetelepta/work/neo4j-llm/data/Annual Report 2023.pdf', 'page': 52}
===
2023-ING-Groep-NV-annual-report.pdf
{'source': '/Users/stetelepta/work/neo4j-llm/data/2023-ING-Groep-NV-annual-report.pdf', 'page': 207}
{'source': '/Users/stetelepta/work/neo4j-llm/data/2023-ING-Groep-NV-annual-report.pdf', 'page': 208}
{'source': '/Users/stetelepta/work/neo4j-llm/data/2023-ING-Groep-NV-annual-report.pdf', 'page': 209}
error getting json, skip page
CPU times: user 344 ms, sys: 20.9 ms, total: 365 ms
Wall time: 56.6 s


In [15]:
for report in annual_reports:
    print(report['company'])
    for d in report['responses_json']:
        for k, v in d.items():
            if k == "response":
                display(pd.DataFrame([v]).T)
            print(k, v)

Rabobank
page_index 69
page 50


,0
Net interest income,11712
Net fee and commission income,2091
Other results,1602
Total income,15405
Staff costs,5858
Other administrative expenses,1851
Depreciation and amortization,348
Total operating expenses,8057
Gross result,7348
Impairment losses on goodwill and associates,105


response {'Net interest income': 11712, 'Net fee and commission income': 2091, 'Other results': 1602, 'Total income': 15405, 'Staff costs': 5858, 'Other administrative expenses': 1851, 'Depreciation and amortization': 348, 'Total operating expenses': 8057, 'Gross result': 7348, 'Impairment losses on goodwill and associates': 105, 'Impairment charges on financial assets': 727, 'Regulatory levies': 554, 'Operating profit before tax': 5962, 'Income tax': 1585, 'Net profit': 4377, 'Impairment charges on financial assets (in basis points)': 17, 'Cost/income ratio including regulatory levies': '55.9%', 'Underlying cost/income ratio including regulatory levies': '57.4%', 'RoE': '9.1%', 'Total assets': 613.8, 'Private sector loan portfolio': 434.0, 'Deposits from customers': 391.4, 'Number of internal employees (in FTEs)': 40467, 'Number of external employees (in FTEs)': 8665, 'Total number of employees (in FTEs)': 49132}
page_index 70
page 51


,0
financial_KPIs,"{'Net Interest Income': 'EUR 11,712 million', ..."
summary,The financial performance of Rabobank in 2023 ...
performance,Better than expected


response {'financial_KPIs': {'Net Interest Income': 'EUR 11,712 million', 'Net Fee and Commission Income': 'EUR 2,091 million', 'Other Results': 'EUR 1,602 million', 'Operating Expenses': 'EUR 8,057 million', 'Operating Profit before tax': 'EUR 5,962 million', 'Taxes': 'EUR 1,585 million', 'Net Profit Retained': 'EUR 3,575 million'}, 'summary': 'The financial performance of Rabobank in 2023 showed an increase in net interest income, stable net fee and commission income, and higher other results. Operating expenses also increased, leading to an operating profit before tax of EUR 5,962 million. Taxes amounted to EUR 1,585 million, resulting in a net profit retained of EUR 3,575 million.', 'performance': 'Better than expected'}
page_index 71
page 52


...

response {'financial_KPIs': {'Total staff costs': 'EUR 5,858 million', 'Total other administrative expenses': 'EUR 1,851 million', 'Depreciation and amortization': 'EUR 348 million', 'Impairment charges on financial assets': 'EUR 727 million', 'Non-performing loans (NPL)': 'EUR 9.0 billion', 'Regulatory levies': 'EUR 554 million', 'Total assets (12-31-2023)': 'EUR 613.8 billion', 'Total liabilities (12-31-2023)': 'EUR 564.2 billion', 'Equity (12-31-2023)': 'EUR 49.6 billion'}, 'summary': 'In 2023, total staff costs, total other administrative expenses, depreciation and amortization, impairment charges on financial assets, non-performing loans, and regulatory levies all increased compared to 2022. However, regulatory levies decreased. The total assets, total liabilities, and equity also increased in 2023 compared to 2022. Overall, the performance seems to be in line with expectations as the increase in costs and liabilities is balanced by an increase in assets and equity.'}
ING
page_ind

,0
financial_KPIs,"{'Total interest income': '52,227', 'Total exp..."
summary,The financial performance of the company in 20...


response {'financial_KPIs': {'Total interest income': '52,227', 'Total expenses': '12,084', 'Net interest income': '15,976', 'Net result': '7,521', 'Net fee and commission income': '3,595', 'Total income': '22,575'}, 'summary': 'The financial performance of the company in 2023 shows an increase in total interest income, net interest income, net result, net fee and commission income, and total income compared to the previous years. This indicates a positive performance that is better than expected.'}
page_index 318
page 208


,0
financial_KPIs,{'Net result (before non-controlling interests...
summary,The financial performance of the company has s...
performance,better than expected


response {'financial_KPIs': {'Net result (before non-controlling interests)': {'2023': 7521, '2022': 3777, '2021': 4905}, 'Total comprehensive income': {'2023': 8507, '2022': 636, '2021': 3096}}, 'summary': 'The financial performance of the company has significantly improved in 2023 compared to the previous years, with a notable increase in net result and total comprehensive income.', 'performance': 'better than expected'}
